<a href="https://colab.research.google.com/github/Propa-Punam/Wifi-RSS-Crowdsensing/blob/main/normalized_vector_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.utils import shuffle
import os
import sys
from datetime import datetime

# Create a timestamped filename for the output
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"/content/drive/My Drive/kohonen_output_{timestamp}.txt"

# Redirect stdout to both file and console
class Logger:
    def __init__(self, filename):
        self.terminal = sys.stdout
        self.log = open(filename, 'w')

    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)
        self.log.flush()

    def flush(self):
        self.terminal.flush()
        self.log.flush()

# Mount Google Drive
drive.mount('/content/drive')

# Redirect stdout
sys.stdout = Logger(output_file)

# Define file paths
train_path = '/content/drive/My Drive/train.csv'
test_path = '/content/drive/My Drive/test.csv'

# Load data from Google Drive
print("Loading training data from:", train_path)
train_df = pd.read_csv(train_path)
print("Loading test data from:", test_path)
test_df = pd.read_csv(test_path)
print("Removing last 3 rows from training data...")
train_df = train_df.iloc[:-3]  # remove l1, l2, and l3

# Ensure 'room' column is string type and stripped
print("Converting 'room' column to string and stripping whitespace...")
train_df['room'] = train_df['room'].astype(str).str.strip()
test_df['room'] = test_df['room'].astype(str).str.strip()

# Shuffle the training data
print("Shuffling training data...")
train_df = shuffle(train_df, random_state=42).reset_index(drop=True)

# Extract RSSI values and labels
print("Extracting RSSI values and labels...")
train_rssi = train_df.iloc[:, 2:].values
train_labels = train_df['room'].values
test_rssi = test_df.iloc[:, 2:].values
test_labels = test_df['room'].values
print(f"Training RSSI shape: {train_rssi.shape}")
print(f"Test RSSI shape: {test_rssi.shape}")

# Define zones and mapping
zones = ['203', '204']
zone_to_idx = {zone: idx for idx, zone in enumerate(zones)}
print("Defined zones:", zones)
print("Zone to index mapping:", zone_to_idx)

# Parameters
n_aps = train_rssi.shape[1]
n_zones = len(zones)
learning_rate = 0.01
n_iterations = 100
print(f"Number of APs: {n_aps}")
print(f"Number of zones: {n_zones}")
print(f"Learning rate: {learning_rate}")
print(f"Number of iterations: {n_iterations}")

# Step 1: Initialize Weights with Normalized Means
print("\n--- Step 1: Initializing Weights ---")
csv_path = "/content/drive/My Drive/room_wise_means.csv"
print("Loading room-wise means from:", csv_path)
df_means = pd.read_csv(csv_path)
print("Removing last 3 rows from means data...")
df_means = df_means.iloc[:-3]

# Extract zones and RSSI values
zones = df_means.iloc[:, 0].tolist()
weights = df_means.iloc[:, 1:].to_numpy()
print("Extracted zones:", zones)
print("Initial weights shape:", weights.shape)

# Normalize weights
normalized_weights = np.zeros_like(weights)
for j in range(n_zones):
    print(f"\nNormalizing weights for zone {zones[j]} (Neuron {j+1}):")
    print("  Original weights:", weights[j])
    # Step-by-step magnitude calculation
    squared_sum = 0
    for ap in range(n_aps):
        squared_value = weights[j, ap] ** 2
        print(f"    rss_{ap+1}^2 = {weights[j, ap]} * {weights[j, ap]} = {squared_value}")
        squared_sum += squared_value
    print(f"  Sum of squared values: {squared_sum}")
    magnitude = np.sqrt(squared_sum)
    print(f"  Magnitude (sqrt(sum)): {magnitude}")

    # Normalize
    if magnitude != 0:
        for ap in range(n_aps):
            normalized_weights[j, ap] = weights[j, ap] / magnitude
            print(f"    Normalized rss_{ap+1} = {weights[j, ap]} / {magnitude} = {normalized_weights[j, ap]}")
    else:
        normalized_weights[j] = weights[j]
        print("    Magnitude is 0, keeping original values")
    print("  Normalized weights:", normalized_weights[j])

weights = normalized_weights.copy()
zone_to_idx = {zone: i for i, zone in enumerate(zones)}
print("\nZone to Index Mapping:", zone_to_idx)
print("\nInitialized Normalized Weight Matrix:")
print(weights)

# Step 2: Train the Kohonen Layer
print("\n--- Step 2: Training the Kohonen Layer ---")
for iteration in range(n_iterations):
    print(f"\nIteration {iteration + 1}:")
    for i in range(len(train_rssi)):
        input_vector = train_rssi[i]
        true_zone = train_labels[i]
        print(f"\nTraining Sample {i + 1} (True Zone: {true_zone}):")
        print(f"  Original Input Vector: {input_vector}")

        # Normalize input vector
        print("  Normalizing input vector:")
        squared_sum = 0
        for ap in range(n_aps):
            squared_value = input_vector[ap] ** 2
            print(f"    rss_{ap+1}^2 = {input_vector[ap]} * {input_vector[ap]} = {squared_value}")
            squared_sum += squared_value
        print(f"  Sum of squared values: {squared_sum}")
        input_magnitude = np.sqrt(squared_sum)
        print(f"  Magnitude (sqrt(sum)): {input_magnitude}")

        normalized_input = np.zeros_like(input_vector, dtype=float)
        if input_magnitude != 0:
            for ap in range(n_aps):
                normalized_input[ap] = input_vector[ap] / input_magnitude
                print(f"    Normalized rss_{ap+1} = {input_vector[ap]} / {input_magnitude} = {normalized_input[ap]}")
        else:
            normalized_input = input_vector
            print("    Magnitude is 0, keeping original values")
        print(f"  Normalized Input Vector: {normalized_input}")

        # Compute VAL_j
        val_j = np.zeros(n_zones)
        for j in range(n_zones):
            print(f"\n  Calculating VAL_{j + 1} for Zone {zones[j]}:")
            sum_products = 0
            for ap in range(n_aps):
                product = normalized_input[ap] * weights[j, ap]
                print(f"    rss_{ap+1} * w_{ap+1}{j+1} = {normalized_input[ap]} * {weights[j, ap]} = {product}")
                sum_products += product
            val_j[j] = sum_products
            print(f"  VAL_{j + 1} = {val_j[j]}")

        # Find the winner
        winner_idx = np.argmax(val_j)
        winner_zone = zones[winner_idx]
        print(f"  Winner: L{winner_idx + 1} (Zone {winner_zone})")

        # Update weights
        old_weights = weights[winner_idx].copy()
        print(f"\n  Updating weights for Neuron L{winner_idx + 1} (Zone {winner_zone}):")
        print("    Old weights:", old_weights)
        weight_update = learning_rate * (normalized_input - old_weights)
        for ap in range(n_aps):
            print(f"    Update for w_{ap+1}{winner_idx+1}: {learning_rate} * ({normalized_input[ap]} - {old_weights[ap]}) = {weight_update[ap]}")
        weights[winner_idx] += weight_update
        print("    Weights after update:", weights[winner_idx])

        # Re-normalize updated weights
        print("    Re-normalizing updated weights:")
        squared_sum = 0
        for ap in range(n_aps):
            squared_value = weights[winner_idx, ap] ** 2
            print(f"      w_{ap+1}{winner_idx+1}^2 = {weights[winner_idx, ap]} * {weights[winner_idx, ap]} = {squared_value}")
            squared_sum += squared_value
        print(f"    Sum of squared values: {squared_sum}")
        new_magnitude = np.sqrt(squared_sum)
        print(f"    New Magnitude: {new_magnitude}")

        if new_magnitude != 0:
            for ap in range(n_aps):
                weights[winner_idx, ap] = weights[winner_idx, ap] / new_magnitude
                print(f"      Normalized w_{ap+1}{winner_idx+1} = {weights[winner_idx, ap]} / {new_magnitude} = {weights[winner_idx, ap]}")
        print("    Final normalized weights:", weights[winner_idx])

        # Show updates
        print(f"\n  Updates:")
        for j in range(n_zones):
            if j == winner_idx:
                print(f"    Neuron L{j + 1} (Zone {zones[j]}): Updated")
                print(f"      Old Weights: {old_weights}")
                print(f"      New Weights: {weights[j]}")
            else:
                print(f"    Neuron L{j + 1} (Zone {zones[j]}): Not Updated")
                print(f"      Weights: {weights[j]}")

        print("\nUpdated Weight Matrix:")
        print(weights)

print("\nFinal Trained Weight Matrix:")
print(weights)

# Step 3: Test the Model
print("\n--- Step 3: Testing the Model ---")
predictions = []
for i, input_vector in enumerate(test_rssi):
    print(f"\nTest Sample {i + 1} (True Zone: {test_labels[i]}):")
    print(f"  Original Input Vector: {input_vector}")

    # Normalize test input vector
    print("  Normalizing input vector:")
    squared_sum = 0
    for ap in range(n_aps):
        squared_value = input_vector[ap] ** 2
        print(f"    rss_{ap+1}^2 = {input_vector[ap]} * {input_vector[ap]} = {squared_value}")
        squared_sum += squared_value
    print(f"  Sum of squared values: {squared_sum}")
    input_magnitude = np.sqrt(squared_sum)
    print(f"  Magnitude (sqrt(sum)): {input_magnitude}")

    normalized_input = np.zeros_like(input_vector, dtype=float)
    if input_magnitude != 0:
        for ap in range(n_aps):
            normalized_input[ap] = input_vector[ap] / input_magnitude
            print(f"    Normalized rss_{ap+1} = {input_vector[ap]} / {input_magnitude} = {normalized_input[ap]}")
    else:
        normalized_input = input_vector
        print("    Magnitude is 0, keeping original values")
    print(f"  Normalized Input Vector: {normalized_input}")

    # Compute VAL_j
    val_j = np.zeros(n_zones)
    for j in range(n_zones):
        print(f"\n  Calculating VAL_{j + 1} for Zone {zones[j]}:")
        sum_products = 0
        for ap in range(n_aps):
            product = normalized_input[ap] * weights[j, ap]
            print(f"    rss_{ap+1} * w_{ap+1}{j+1} = {normalized_input[ap]} * {weights[j, ap]} = {product}")
            sum_products += product
        val_j[j] = sum_products
        print(f"  VAL_{j + 1} = {val_j[j]}")

    winner_idx = np.argmax(val_j)
    predicted_zone = zones[winner_idx]
    predictions.append(predicted_zone)
    print(f"  Predicted Zone: {predicted_zone}")

# Step 4: Calculate Accuracy
print("\n--- Step 4: Results ---")
correct = 0
print("Calculating accuracy...")
for i, (pred, true) in enumerate(zip(predictions, test_labels)):
    print(f"  Sample {i + 1}: Predicted: {pred}, True: {true}")
    if pred == true:
        correct += 1
        print(f"    Match: Correct prediction")
    else:
        print(f"    No match: Incorrect prediction")
total = len(test_labels)
accuracy = correct / total * 100
print(f"\nAccuracy: {accuracy:.2f}% ({correct}/{total})")

# Save the final trained weights
print("\nSaving final trained weights...")
weights_df = pd.DataFrame(weights)
weights_df.insert(0, 'Room', zones)
output_weights_path = f"/content/drive/My Drive/trained_weights_{timestamp}.csv"
weights_df.to_csv(output_weights_path, index=False)
print(f"Trained weights saved to: {output_weights_path}")

# Restore original stdout
sys.stdout = sys.__stdout__
print(f"Output has been saved to {output_file}")
print(f"Trained weights have been saved to {output_weights_path}")